# Project 1, Part 1, Executive Questions: Sales Specific Questions

University of California, Berkeley
Master of Information and Data Science (MIDS) program
w205 - Fundamentals of Data Engineering

Student: Sophie Yeh

Year: 2022

Semester: Spring

Section: 8


# Included Modules and Packages

Code cell containing your includes for modules and packages

In [15]:
!pip install plotly
import math
import numpy as np
import pandas as pd
import psycopg2
import plotly.express as px


# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [16]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
#connect to postgres database
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
# create cursor for connections
cursor = connection.cursor()

# Instructions for Specific Questions

For each specific quesion, you must write 1 and only 1 query that answers that specific question.  The results of your query must be displayed in a Pandas dataframe as demonstrated in the labs

All output should be sorted.

For store name, use the city name the store is located in.

When sorting by store name, sort in alphabetical order.

For month, use the full month name: January, February, March, etc.

When sorting by month, sort in calendar order: January, February, March, etc.  You may find it useful to include a month number to help with the sort.  It's ok to leave it in the output.

For day of week, use the full day of week name: Sunday, Monday, Tuesday, etc.

When sorting by day of week, sort in day of week order: Sunday, Monday, Tuesday, etc. You may find it usefule to include a day of week number to help with the sort.  It's ok to leave it in the output.


# 1.1.1 Total Sales as a Dollar Amount for all of AGM

In [17]:
def agm_total_sales():
    # Total Sales as a Dollar Amount for all of AGM
    rollback_before_flag = True
    rollback_after_flag = True
    query = """
    SELECT SUM(total_amount) as total_amount
    FROM sales
    """
    return  my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
agm_total_sales().style.set_caption("Total Sales as a Dollar Amount for all of AGM")

,total_amount
0,98739408


# 1.1.2 Total Sales as a Dollar Amount by Store

Sort by store name in alphabetical order

In [18]:
def sales_by_store():
    # Total Sales as a Dollar Amount by Store
    rollback_before_flag = True
    rollback_after_flag = True
    query = """

    SELECT s.city as store, sum(sa.total_amount) as total_amount
    FROM stores as s
        JOIN sales as sa
            ON s.store_id = sa.store_id
    GROUP BY store
    ORDER BY total_amount

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
sales_by_store().style.set_caption('Total Sales as a Dollar Amount by Store')

,store,total_amount
0,Nashville,14573172
1,Miami,17692404
2,Dallas,19408260
3,Seattle,22024512
4,Berkeley,25041060


# 1.1.3 Total Sales as a Dollar Amount by Month

Sort by month full name in calendar order

In [19]:
def sales_by_month():
    # Total Sales as a Dollar Amount by Month
    rollback_before_flag = True
    rollback_after_flag = True
    query = """
    SELECT EXTRACT(MONTH FROM sale_date) as month_num, 
           to_char(sale_date, 'MONTH') as month_name, 
           sum(total_amount) as total_amount
    FROM sales
    GROUP BY month_num, month_name
    ORDER BY month_num, month_name

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
sales_by_month().style.set_caption('Total Sales as a Dollar Amount by Month')

,month_num,month_name,total_amount
0,1,JANUARY,7803828
1,2,FEBRUARY,7574280
2,3,MARCH,8779620
3,4,APRIL,8251284
4,5,MAY,7977840
5,6,JUNE,8124108
6,7,JULY,7993044
7,8,AUGUST,9029808
8,9,SEPTEMBER,7578960
9,10,OCTOBER,8895108


There is no consistent dollar amount sold by month. However, we can take note of peak in sales during March, August, and October. These months fall in Q1, Q2, and Q3 respectively.

# 1.1.4 Total Sales as a Dollar Amount by Store and Month

Sort by store name in alphabetical order, then by month full name in calendar order

In [20]:
def sales_by_store_month():
    # Total Sales as a Dollar Amount by Store and Month
    rollback_before_flag = True
    rollback_after_flag = True
    query = """

    SELECT s.city as store, 
           EXTRACT(MONTH FROM sa.sale_date) as month_num, 
           to_char(sa.sale_date, 'MONTH') as month_name, 
           sum(sa.total_amount) as total_amount
    FROM stores as s
        JOIN sales as sa
            ON s.store_id = sa.store_id
    GROUP BY store, month_num, month_name
    ORDER BY store, month_num, month_name

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

sales_by_store_month().style.set_caption('Total Sales as a Dollar Amount by Store and Month')


,store,month_num,month_name,total_amount
0,Berkeley,1,JANUARY,1988904
1,Berkeley,2,FEBRUARY,1930272
2,Berkeley,3,MARCH,2224500
3,Berkeley,4,APRIL,2092056
4,Berkeley,5,MAY,2019264
5,Berkeley,6,JUNE,2065140
6,Berkeley,7,JULY,2034708
7,Berkeley,8,AUGUST,2286732
8,Berkeley,9,SEPTEMBER,1922256
9,Berkeley,10,OCTOBER,2248008


Breaking up total amount by month into stores show that all stores follow the same pattern.

# 1.1.5 Total Sales as a Dollar Amount by Day of Week

Sort by day of week in day of week order.

In [21]:
def sales_by_dow():
    # Total Sales as a Dollar Amount by Day of Week
    rollback_before_flag = True
    rollback_after_flag = True
    query = """

    SELECT 
        EXTRACT(dow FROM sale_date) as dow, 
        to_char(sale_date, 'Day') as day_of_week, 
        sum(total_amount) as total_amount
    FROM sales
    GROUP BY dow, day_of_week
    ORDER By dow, day_of_week

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

sales_by_dow().style.set_caption('Total Sales as a Dollar Amount by Day of Week')


,dow,day_of_week,total_amount
0,0,Sunday,18589068
1,1,Monday,13167720
2,2,Tuesday,6895332
3,3,Wednesday,13952556
4,4,Thursday,13834644
5,5,Friday,12878628
6,6,Saturday,19421460


Sales are highest on Saturday followed by Sunday. There is a dip in sales on Tuesdays.

# 1.1.6 Total Sales as a Dollar Amount by Store and Day of Week

Sort by store name in alphabetical order, then by day of week in day of week order.

In [22]:
def sales_by_store_dow():
    # Total Sales as a Dollar Amount by Store and Day of Week
    rollback_before_flag = True
    rollback_after_flag = True
    query = """

    SELECT 
        s.city as store,
        EXTRACT(dow FROM sa.sale_date) as dow, 
        to_char(sa.sale_date, 'Day') as day_of_week, 
        sum(sa.total_amount) as total_amount
    FROM stores as s
        JOIN sales as sa
            ON s.store_id = sa.store_id
    GROUP BY store, dow, day_of_week
    ORDER By store, dow, day_of_week

    """ 
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

sales_by_store_dow().style.set_caption('Total Sales as a Dollar Amount by Store and Day of Week')

,store,dow,day_of_week,total_amount
0,Berkeley,0,Sunday,4694640
1,Berkeley,1,Monday,3340116
2,Berkeley,2,Tuesday,1752036
3,Berkeley,3,Wednesday,3546144
4,Berkeley,4,Thursday,3507660
5,Berkeley,5,Friday,3273240
6,Berkeley,6,Saturday,4927224
7,Dallas,0,Sunday,3650748
8,Dallas,1,Monday,2602980
9,Dallas,2,Tuesday,1352760


Breaking up total amount by day of the week into stores show that all stores follow the same pattern.

# 1.1.7 Total Number of Sales for all of AGM

In [23]:
def sales_ct_agm():
    rollback_before_flag = True
    rollback_after_flag = True
    query = """

    SELECT COUNT(total_amount) as count_sales
    FROM sales

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
sales_ct_agm().style.set_caption('Total Number of Sales for all of AGM')

,count_sales
0,1537617


# 1.1.8 Total Number of Sales by Store

Sort by store name in alphabetical order

In [24]:
def sales_ct_store():
    rollback_before_flag = True
    rollback_after_flag = True
    query = """

    SELECT s.city as store, COUNT(sa.sale_id) as sale_count
    FROM stores as s
        JOIN sales as sa
            ON s.store_id = sa.store_id
    GROUP BY store
    ORDER BY store

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
sales_ct_store().style.set_caption('Total Number of Sales by Store')

,store,sale_count
0,Berkeley,390375
1,Dallas,302120
2,Miami,275074
3,Nashville,227721
4,Seattle,342327


Number of sales by store has the same ranking as total dollar amount by store.

# 1.1.9 Average Dollar Amount per Sale 

In [25]:
def avg_amount_per_sale():
    rollback_before_flag = True
    rollback_after_flag = True
    query = """

    SELECT AVG(total_amount) as avg_amt
    FROM sales

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
avg_amount_per_sale().style.set_caption('Average Dollar Amount per Sale')

,avg_amt
0,64.215866


# 1.1.10 Average Dollar Amount per Sale by Store

Sort by store name in alphabetical order

In [26]:
def avg_amount_per_sale_store():
    rollback_before_flag = True
    rollback_after_flag = True
    query = """

    SELECT s.city as store, AVG(sa.total_amount) as avg_amt
    FROM stores as s
        JOIN sales as sa
            ON s.store_id = sa.store_id
    GROUP BY store
    ORDER BY store

    """
    return my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
avg_amount_per_sale_store().style.set_caption('Average Dollar Amount per Sale by Store')

,store,avg_amt
0,Berkeley,64.146167
1,Dallas,64.240236
2,Miami,64.318707
3,Nashville,63.995732
4,Seattle,64.337642


# 1.1.11 The Executives have also asked you to provide your best example of a data visualization for one of the above queries

You may choose any one of the above queries. 

It's best to select the one that you think will provide the most interesting data visualization.  

You are allowed to add a pivot as demonstrated in the labs if it is needed for your data visualization. 

In [28]:
def fig_sales_by_dow():
    fig = px.line(sales_by_dow(), x='day_of_week', y='total_amount',
                 title='Total Sales as a Dollar Amount by Day of Week',
                 labels={'day_of_week': 'day of week',
                        'total_amount':'total amount USD'},
                 template='plotly_white',
                 markers=True)
    fig.show()
    return
fig_sales_by_dow()